In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense, Embedding, GRU, LSTM
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [ ]:
result=pd.read_csv('/content/preprocessed_ade.csv')

In [ ]:
result

,Unnamed: 0,observation,labels
0,12606,early phase ii semi double blind study of the ...,0
1,685,cessation of mtx therapy led to complete regre...,0
2,5127,cutaneous biopsy showed hyphae and round inclu...,0
3,3158,she was treated with acyclovir and subsequentl...,1
4,10685,cisplatin was substituted and the patient achi...,0
...,...,...,...
23511,982,ticlopidine induced aplastic anemia two new c...,1
23512,14487,the risks of sumatriptan administration in pat...,0
23513,4680,however the spectrum of hosts and clinical p...,0
23514,2588,antibiotic treatment should be continued,0


In [ ]:
import gensim
import os
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [ ]:
result.drop('Unnamed: 0' , axis = 1, inplace = True)

In [ ]:
result

,observation,labels
0,early phase ii semi double blind study of the ...,0
1,cessation of mtx therapy led to complete regre...,0
2,cutaneous biopsy showed hyphae and round inclu...,0
3,she was treated with acyclovir and subsequentl...,1
4,cisplatin was substituted and the patient achi...,0
...,...,...
23511,ticlopidine induced aplastic anemia two new c...,1
23512,the risks of sumatriptan administration in pat...,0
23513,however the spectrum of hosts and clinical p...,0
23514,antibiotic treatment should be continued,0


In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-06-24 17:09:14--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.76.102
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.76.102|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
url="https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [ ]:
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url, binary=True) 

In [ ]:
docs_vectors = pd.DataFrame() # creating empty final dataframe
stopwords = nltk.corpus.stopwords.words('english') # removing stop words  
for doc in result['observation'].str.lower().str.replace('[^a-z ]', ''): # looping through each document and cleaning it
    temp = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
    for word in doc.split(' '): # looping through each word of a single document and spliting through space
        if word not in stopwords: # if word is not present in stopwords then (try)
            try:
                word_vec = embeddings[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                temp = temp.append(pd.Series(word_vec), ignore_index = True) # if word is present then append it to temporary dataframe
            except:
                pass
    doc_vector = temp.mean() # take the average of each column(w0, w1, w2,........w300)
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) # append each document value to the final dataframe
docs_vectors.shape

(23516, 300)

In [ ]:
docs_vectors['labels'] = result['labels'] 

In [ ]:
docs_vectors = docs_vectors.dropna()

In [ ]:
docs_vectors

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,labels
0,-0.050695,-0.013357,-0.008664,0.066528,-0.052307,-0.042175,0.077194,-0.106771,0.094014,0.025045,-0.002279,-0.073015,-0.062754,0.023458,-0.113424,0.005196,-0.003326,0.046899,0.092621,-0.086620,-0.030538,-0.061499,-0.107239,-0.073883,-0.008952,0.003123,-0.046076,0.072083,0.063965,0.034098,-0.001490,-0.015025,0.024556,-0.051412,-0.079405,-0.059469,0.018873,-0.052607,0.112142,-0.071477,...,-0.039042,-0.066981,0.039108,0.031909,0.104273,-0.026176,-0.130332,-0.123133,0.029694,0.054385,0.017690,0.007355,-0.070262,0.055140,0.007670,-0.037689,-0.070933,-0.019201,0.056788,0.044965,0.039923,0.023499,0.012102,0.052272,-0.067769,0.005969,-0.063703,0.014497,0.013255,-0.005880,0.069450,-0.066691,-0.055615,-0.068281,-0.005549,-0.106692,0.031687,-0.025411,0.035197,0
1,-0.003021,0.127523,0.022970,0.078756,-0.108480,-0.006226,-0.025464,-0.258464,0.102407,-0.010974,-0.041590,-0.183228,0.019145,0.065308,0.030192,0.245097,0.080068,-0.014318,0.003296,-0.170492,0.016622,0.125977,-0.055054,0.191996,-0.136230,0.046834,-0.257161,-0.001546,0.073795,-0.008087,0.038411,-0.253988,-0.172902,0.072103,-0.274129,-0.037720,-0.071818,-0.001750,0.019694,-0.058512,...,0.023560,-0.005086,-0.016195,0.122925,0.153564,0.092306,-0.107666,-0.094686,-0.038452,0.129679,0.189931,-0.090637,-0.015818,0.167480,0.069153,-0.120809,0.080282,0.039800,0.011545,0.092285,0.134562,0.054606,-0.062337,-0.047445,-0.135681,0.168050,0.019450,0.060410,-0.014730,-0.113851,0.076541,0.077482,-0.045329,-0.001383,-0.022359,-0.056244,0.130605,0.129150,0.039032,0
2,-0.029696,0.117454,0.070279,0.049297,-0.143677,0.020880,-0.034909,-0.225763,0.071650,0.107422,-0.133434,-0.185225,-0.064337,0.098145,-0.072820,0.187689,0.063008,0.230735,-0.073231,0.010345,-0.069647,-0.056596,-0.062522,0.020130,-0.188754,-0.137140,-0.148304,0.077767,-0.025036,-0.118508,-0.092396,0.002475,0.010717,-0.007025,-0.138278,0.005085,0.043518,0.064731,0.043790,0.038613,...,0.048273,-0.010312,-0.112371,0.046531,0.120449,0.060059,-0.127644,0.040558,-0.075150,-0.036205,0.173329,-0.076527,-0.005085,0.192204,-0.018815,-0.092661,0.006947,0.019054,0.138184,0.096868,0.022367,0.050756,-0.051164,-0.029707,-0.083724,0.039218,-0.012851,-0.146107,0.018256,-0.018488,-0.023948,0.089930,0.033869,-0.042969,0.057861,-0.091569,-0.049128,0.084306,0.066426,0
3,-0.034995,0.044120,0.062256,0.079346,-0.163993,-0.040091,-0.057486,-0.120919,0.014265,0.075091,-0.192383,-0.232945,0.041138,0.021345,-0.151341,0.085903,-0.082441,0.236258,0.020050,0.091936,0.004220,0.075405,0.031786,-0.028285,-0.077676,-0.140102,-0.061558,0.049909,-0.002860,0.038225,-0.122105,-0.236049,-0.023071,0.013157,-0.068324,0.045576,0.005197,0.019531,-0.029465,-0.038295,...,-0.009234,-0.005153,-0.133806,0.153460,0.087899,0.072405,-0.104771,-0.013811,0.003383,0.105730,0.146362,-0.050642,-0.025257,0.165414,-0.028983,-0.061070,0.136161,0.002921,0.026228,-0.047032,0.073661,0.012591,-0.096392,-0.066232,-0.169316,0.125610,0.111904,-0.083758,0.105756,-0.125488,0.016497,-0.001325,0.106097,0.092913,0.194301,-0.168597,0.067226,0.183340,-0.034546,1
4,0.015993,0.063070,-0.002374,0.145820,-0.082438,0.000461,0.064053,-0.192383,0.125637,0.018826,-0.172716,-0.213867,0.005659,0.001294,0.017849,0.249525,0.088019,0.018311,-0.027913,-0.134142,0.034424,0.093031,-0.089423,0.083279,-0.061584,-0.042616,-0.188856,-0.040602,0.014445,0.056342,0.032718,-0.126126,-0.226671,-0.081546,-0.199599,-0.025684,0.005062,0.018643,0.045939,-0.028630,...,-0.041456,-0.042101,-0.080397,0.130995,0.052138,0.089925,-0.214205,-0.121718,-0.065077,0.069255,0.155545,-0.040944,0.047689,0.095893,-0.055393,-0.195258,-0.037672,0.029673,0.011854,0.120734,0.132958,-0.021444,-0.061561,-0.047670,-0.111681,0.130615,-0.031660,0.005324,0.039551,-0.141086,0.047343,0.018546

In [ ]:
validation_data = docs_vectors.sample(frac = 0.2)
docs_vectors=docs_vectors.drop(validation_data.index) 

In [ ]:
validation_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,labels
15265,0.012791,0.084262,0.028987,0.078275,-0.151117,-0.047687,-0.011377,-0.199054,0.108130,0.135742,-0.038281,-0.209519,0.003735,-0.015405,-0.058685,0.165201,-0.015845,0.137793,0.053732,-0.064978,-0.055367,0.071582,-0.054956,0.087738,-0.035095,-0.069922,-0.025568,0.117480,-0.077795,0.118774,0.006146,-0.120764,-0.050839,-0.059509,-0.036829,0.035977,-0.012891,0.016296,-0.017908,0.027661,...,-0.032983,-0.078162,-0.044568,0.058362,0.068650,0.056914,-0.057497,-0.159131,-0.058247,0.004016,0.161501,0.074414,-0.008984,0.095612,0.040614,-0.111981,0.040112,0.025394,0.081128,0.082593,0.025566,-0.055554,-0.054932,-0.017975,-0.120068,0.141943,-0.001819,0.041910,0.042971,0.047988,-0.035193,-0.033740,-0.025098,0.014246,0.034201,-0.155176,-0.006445,0.145923,0.072852,1
8002,-0.089577,0.045266,0.057151,0.076527,-0.149708,0.078513,0.039173,-0.121767,0.072685,-0.050559,-0.087025,-0.144942,-0.002916,0.033436,-0.106495,0.061730,0.018000,0.115087,-0.017079,-0.027033,-0.055631,-0.099454,0.042702,0.141790,-0.031422,-0.086526,-0.080766,0.072332,0.003296,-0.013694,-0.101873,-0.059781,0.018200,-0.068648,-0.184737,-0.054310,0.011697,-0.025646,0.001060,0.013686,...,-0.068160,-0.056175,-0.014393,0.055586,0.076299,0.031894,-0.051572,-0.111373,-0.217934,-0.023804,0.158281,0.026334,-0.031203,-0.004705,-0.076849,-0.029266,0.019725,-0.016975,0.063077,0.089189,0.096774,0.000803,-0.063207,0.056560,-0.045915,0.038641,0.027460,0.019942,0.059501,-0.070634,-0.021706,-0.099293,-0.036843,-0.003777,0.100604,-0.044378,-0.006001,0.090959,0.029261,0
21718,-0.000244,0.122955,0.074524,0.061340,-0.047974,0.009277,0.037735,-0.153503,0.078125,0.112793,-0.044586,-0.160034,0.039623,0.144104,-0.044312,0.145752,0.022156,0.282837,-0.028564,-0.199493,-0.085205,0.006409,0.123810,-0.053787,-0.267120,-0.172241,-0.004150,0.120636,0.031902,-0.033798,0.020752,-0.052673,0.136703,0.027962,-0.170593,-0.141113,0.048950,-0.061752,0.127075,0.119141,...,-0.080933,-0.056030,0.027832,0.148865,0.168335,-0.211182,-0.081192,0.094238,-0.091675,-0.132080,0.259583,0.001709,0.085198,0.202026,0.103844,0.016525,-0.039536,0.012573,0.125000,0.098145,-0.009094,0.008179,0.102188,-0.153687,-0.212463,0.008667,0.133057,0.046753,0.182983,0.088539,-0.041992,0.014618,0.044678,0.008514,-0.009720,-0.037354,0.276611,0.268677,0.114075,1
11924,-0.073042,0.000433,0.055775,0.075040,-0.015714,0.032138,-0.008406,-0.106179,0.115767,-0.016185,-0.039112,-0.110074,0.073864,0.074419,-0.052056,0.064309,0.020547,0.092077,-0.020595,-0.090898,0.097168,-0.021618,-0.027727,0.066847,-0.057989,-0.046600,-0.087569,0.074474,-0.070756,0.071777,-0.042788,-0.087497,-0.064686,-0.083196,-0.165672,-0.000488,0.021440,-0.033114,-0.024711,-0.010589,...,0.037065,-0.100630,0.012532,0.037093,0.039365,-0.001631,-0.084029,-0.126909,-0.073120,0.090532,0.097895,0.024392,0.058627,0.112416,-0.030751,-0.091986,0.031994,0.004023,-0.005621,0.064797,0.052024,0.060347,0.027180,-0.054255,-0.059282,0.061129,0.026176,-0.003235,0.025723,0.025743,-0.051891,-0.022705,0.062933,0.005576,0.020208,-0.123280,-0.015148,0.102800,0.070157,1
9999,0.058792,0.165047,-0.046371,0.018402,-0.061676,-0.038177,0.031616,-0.160995,0.085815,0.058502,-0.120728,-0.122620,-0.080435,0.114204,-0.041298,0.044098,-0.059769,0.082397,-0.035419,0.010612,0.016907,-0.074341,0.006374,-0.010605,-0.035507,-0.117035,-0.117966,0.065430,-0.016411,-0.101624,-0.008057,-0.168274,-0.073288,-0.148315,-0.154755,0.056105,-0.016903,0.026512,-0.005672,-0.059681,...,0.036049,0.048555,0.131393,0.183258,0.117645,-0.001751,-0.201263,-0.092903,-0.053619,0.034531,0.123222,-0.070007,-0.092987,0.127823,-0.059418,-0.077087,0.048370,0.067139,-0.021805,0.167435,0.094505,0.107544,-0.049393,-0.106422,-0.002983,0.128662,0.122818,-0.023041,0.011271,-0.066765

In [ ]:
x_val=validation_data.drop('labels' , axis=1) 
y_val=validation_data['labels']


In [ ]:
x_val

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
15265,0.012791,0.084262,0.028987,0.078275,-0.151117,-0.047687,-0.011377,-0.199054,0.108130,0.135742,-0.038281,-0.209519,0.003735,-0.015405,-0.058685,0.165201,-0.015845,0.137793,0.053732,-0.064978,-0.055367,0.071582,-0.054956,0.087738,-0.035095,-0.069922,-0.025568,0.117480,-0.077795,0.118774,0.006146,-0.120764,-0.050839,-0.059509,-0.036829,0.035977,-0.012891,0.016296,-0.017908,0.027661,...,-0.026331,-0.032983,-0.078162,-0.044568,0.058362,0.068650,0.056914,-0.057497,-0.159131,-0.058247,0.004016,0.161501,0.074414,-0.008984,0.095612,0.040614,-0.111981,0.040112,0.025394,0.081128,0.082593,0.025566,-0.055554,-0.054932,-0.017975,-0.120068,0.141943,-0.001819,0.041910,0.042971,0.047988,-0.035193,-0.033740,-0.025098,0.014246,0.034201,-0.155176,-0.006445,0.145923,0.072852
8002,-0.089577,0.045266,0.057151,0.076527,-0.149708,0.078513,0.039173,-0.121767,0.072685,-0.050559,-0.087025,-0.144942,-0.002916,0.033436,-0.106495,0.061730,0.018000,0.115087,-0.017079,-0.027033,-0.055631,-0.099454,0.042702,0.141790,-0.031422,-0.086526,-0.080766,0.072332,0.003296,-0.013694,-0.101873,-0.059781,0.018200,-0.068648,-0.184737,-0.054310,0.011697,-0.025646,0.001060,0.013686,...,0.003263,-0.068160,-0.056175,-0.014393,0.055586,0.076299,0.031894,-0.051572,-0.111373,-0.217934,-0.023804,0.158281,0.026334,-0.031203,-0.004705,-0.076849,-0.029266,0.019725,-0.016975,0.063077,0.089189,0.096774,0.000803,-0.063207,0.056560,-0.045915,0.038641,0.027460,0.019942,0.059501,-0.070634,-0.021706,-0.099293,-0.036843,-0.003777,0.100604,-0.044378,-0.006001,0.090959,0.029261
21718,-0.000244,0.122955,0.074524,0.061340,-0.047974,0.009277,0.037735,-0.153503,0.078125,0.112793,-0.044586,-0.160034,0.039623,0.144104,-0.044312,0.145752,0.022156,0.282837,-0.028564,-0.199493,-0.085205,0.006409,0.123810,-0.053787,-0.267120,-0.172241,-0.004150,0.120636,0.031902,-0.033798,0.020752,-0.052673,0.136703,0.027962,-0.170593,-0.141113,0.048950,-0.061752,0.127075,0.119141,...,-0.042725,-0.080933,-0.056030,0.027832,0.148865,0.168335,-0.211182,-0.081192,0.094238,-0.091675,-0.132080,0.259583,0.001709,0.085198,0.202026,0.103844,0.016525,-0.039536,0.012573,0.125000,0.098145,-0.009094,0.008179,0.102188,-0.153687,-0.212463,0.008667,0.133057,0.046753,0.182983,0.088539,-0.041992,0.014618,0.044678,0.008514,-0.009720,-0.037354,0.276611,0.268677,0.114075
11924,-0.073042,0.000433,0.055775,0.075040,-0.015714,0.032138,-0.008406,-0.106179,0.115767,-0.016185,-0.039112,-0.110074,0.073864,0.074419,-0.052056,0.064309,0.020547,0.092077,-0.020595,-0.090898,0.097168,-0.021618,-0.027727,0.066847,-0.057989,-0.046600,-0.087569,0.074474,-0.070756,0.071777,-0.042788,-0.087497,-0.064686,-0.083196,-0.165672,-0.000488,0.021440,-0.033114,-0.024711,-0.010589,...,-0.003412,0.037065,-0.100630,0.012532,0.037093,0.039365,-0.001631,-0.084029,-0.126909,-0.073120,0.090532,0.097895,0.024392,0.058627,0.112416,-0.030751,-0.091986,0.031994,0.004023,-0.005621,0.064797,0.052024,0.060347,0.027180,-0.054255,-0.059282,0.061129,0.026176,-0.003235,0.025723,0.025743,-0.051891,-0.022705,0.062933,0.005576,0.020208,-0.123280,-0.015148,0.102800,0.070157
9999,0.058792,0.165047,-0.046371,0.018402,-0.061676,-0.038177,0.031616,-0.160995,0.085815,0.058502,-0.120728,-0.122620,-0.080435,0.114204,-0.041298,0.044098,-0.059769,0.082397,-0.035419,0.010612,0.016907,-0.074341,0.006374,-0.010605,-0.035507,-0.117035,-0.117966,0.065430,-0.016411,-0.101624,-0.008057,-0.168274,-0.073288,-0.148315,-0.154755,0.056105,-0.016903,0.026512,-0.005672,-0.059681,...,-0.068146,0.036049,0.048555,0.131393,0.183258,0.117645,-0.001751,-0.201263,-0.092903,-0.053619,0.034531,0.123222,-0.070007,-0.092987,0.127823,-0.059418,-0.077087,0.048370,0.067139,-0.021805,0.167435,0.094505,0.107544,-0.049393,-0.106422,-0.002983,0.128662

In [ ]:
y_val

15265    1
8002     0
21718    1
11924    1
9999     0
        ..
20385    0
123      0
17213    1
15649    1
7046     0
Name: labels, Length: 4702, dtype: int64

In [ ]:
print(x_val.shape)
print(y_val.shape)

(4702, 300)
(4702,)


In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
x=docs_vectors.drop('labels' , axis=1)
y=docs_vectors['labels']

In [ ]:
print(x.shape)
print(y.shape)

(18806, 300)
(18806,)


In [ ]:
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
smote = SMOTE(random_state = 2021)
x, y = smote.fit_resample(x,y) 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print(x.shape)
print(y.shape)
print(x_val.shape)
print(y_val.shape) 

(26732, 300)
(26732,)
(4702, 300)
(4702,)


In [ ]:
zerocount=0
onecount=0
for i in y:
  if i == 0:
    zerocount+=1
  else:
    onecount+=1
print("no of negative sample in training class is:",zerocount)
print("no of positive sample in training class is:",onecount) 

no of negative sample in training class is: 13366
no of positive sample in training class is: 13366


**DECISION TREE**

In [ ]:
from sklearn.tree import DecisionTreeClassifier


In [ ]:
dt_model= DecisionTreeClassifier(max_depth=5)  

In [ ]:
dt_model.fit(x,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
dt_pred=dt_model.predict(x_val)

In [ ]:
dt_pred

array([1, 1, 0, ..., 1, 1, 1])

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [ ]:
print(classification_report(y_val,dt_pred))

              precision    recall  f1-score   support

           0       0.87      0.62      0.73      3321
           1       0.46      0.77      0.58      1381

    accuracy                           0.67      4702
   macro avg       0.66      0.70      0.65      4702
weighted avg       0.75      0.67      0.68      4702



In [ ]:
print(confusion_matrix(y_val,dt_pred))

[[2074 1247]
 [ 314 1067]]


In [ ]:
#training error 
dt_pred_train = dt_model.predict(x)
accuracy_score(y,dt_pred_train )  


0.7277046236720036

**LOGISTIC REGRESSION** 

In [ ]:
 from sklearn.linear_model import LogisticRegression

In [ ]:
log_model=LogisticRegression(max_iter=1000)

In [ ]:
log_model.fit(x,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
log_pred=log_model.predict(x_val)

In [ ]:
log_pred

array([1, 0, 1, ..., 1, 1, 0])

In [ ]:
print(classification_report(log_pred,y_val))

              precision    recall  f1-score   support

           0       0.77      0.89      0.83      2892
           1       0.76      0.58      0.66      1810

    accuracy                           0.77      4702
   macro avg       0.77      0.74      0.74      4702
weighted avg       0.77      0.77      0.76      4702



In [ ]:
print(confusion_matrix(log_pred,y_val))

[[2567  325]
 [ 754 1056]]


In [ ]:
#calculate training error 
log_y_pred_train = log_model.predict(x) 
accuracy_score(y,log_y_pred_train) 

0.7909621427502619

**NAIVE BAYES MODELS**

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
gnb_model=GaussianNB()


In [ ]:
gnb_model.fit(x,y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
gnb_pred=gnb_model.predict(x_val)

In [ ]:
print(classification_report(y_val,gnb_pred))

              precision    recall  f1-score   support

           0       0.87      0.70      0.77      3321
           1       0.50      0.74      0.60      1381

    accuracy                           0.71      4702
   macro avg       0.68      0.72      0.69      4702
weighted avg       0.76      0.71      0.72      4702



In [ ]:
print(confusion_matrix(y_val,gnb_pred))

[[2316 1005]
 [ 361 1020]]


In [ ]:
#training error 
gnb_train_pred = gnb_model.predict(x)
accuracy_score(y,gnb_train_pred) 

0.7649633398174472

**RANDOM FOREST CLASSIFIER**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc_model=RandomForestClassifier(n_estimators=1000,max_leaf_nodes=50)

In [ ]:
rfc_model.fit(x,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=50, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
rfc_pred=rfc_model.predict(x_val)

In [ ]:
print(classification_report(y_val,rfc_pred))

              precision    recall  f1-score   support

           0       0.88      0.85      0.87      3321
           1       0.67      0.71      0.69      1381

    accuracy                           0.81      4702
   macro avg       0.77      0.78      0.78      4702
weighted avg       0.82      0.81      0.81      4702



In [ ]:
print(confusion_matrix(y_val,rfc_pred))

[[2838  483]
 [ 396  985]]


In [ ]:
#training error 
rfc_train = rfc_model.predict(x)
accuracy_score(y,rfc_train) 

0.8482343259015412

**XG BOOST**

In [ ]:
pip install xgboost 

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb_model=XGBClassifier()

In [ ]:
xgb_model.fit(x,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
xgb_pred = xgb_model.predict(x_val.values)


In [ ]:
print(classification_report(y_val,xgb_pred))

              precision    recall  f1-score   support

           0       0.91      0.82      0.86      3321
           1       0.65      0.80      0.72      1381

    accuracy                           0.82      4702
   macro avg       0.78      0.81      0.79      4702
weighted avg       0.83      0.82      0.82      4702



In [ ]:
print(confusion_matrix(y_val,xgb_pred)) 

[[2735  586]
 [ 280 1101]]


In [ ]:
#training error 
xgb_pred_train = xgb_model.predict(x) 
accuracy_score(y,xgb_pred_train)   

0.871090827472692